In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from sklearn import tree
import pydot
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn import linear_model
from random import seed
from random import random
from random import randrange
from sklearn.tree import DecisionTreeRegressor

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, mean_absolute_error
from string import ascii_letters
import seaborn as sns
from numpy import ndarray
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from sklearn.metrics import r2_score, mean_absolute_error

# Data preprocessing

In [ ]:
train_data = pd.read_csv("/Users/qinghongxu/Documents/MATH895/project/abalone.data.txt",
                          delimiter=',' )

In [ ]:
train_data = train_data[train_data['Height']>0]

In [ ]:
train_data['Age'] = train_data['Rings'] + 1.5

In [ ]:
v = train_data['Age'].values.tolist()
a = [0] * len(v)
for i in range(len(v)):
    if v[i] <= 9.5:
        a[i] = 0
    elif v[i] >12.5:
        a[i] = 2
    else:
        a[i] = 1

In [ ]:
# 3 - category classfication: 2.5<=Age<=9.5, 9.5<Age<=12.5, Age>12.5
train_data['Target'] = a

In [ ]:
train_data['Male'] = (train_data['Sex']=='M').astype(int)
train_data['Female'] = (train_data['Sex']=='F').astype(int)
train_data['Infant'] = (train_data['Sex']=='I').astype(int)

In [ ]:
train_data['Sex'] = LabelEncoder().fit_transform(train_data['Sex'].tolist()) #F=0.0, I=1.0 , M=2.0

  # Basic Data Analysis

In [ ]:
#train_data = train_data.drop('Target', axis = 1)
train_data = train_data.drop('Sex', axis = 1)
#train_data = train_data.drop('Rings', axis = 1)
corr = train_data.corr('kendall')
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = pl.subplots(figsize=(8, 6))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1,square=True, 
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax);

In [ ]:
Ytrain_data = train_data['Target']
#train_data = train_data.drop('Sex',axis = 1)
train_data = train_data.drop('Rings',axis = 1)
train_data = train_data.drop('Age',axis = 1)
train_data = train_data.drop('Target',axis = 1)


In [ ]:
train_data = train_data.drop('Sex',axis = 1)

In [ ]:
# PCA
A = train_data[0:3133].values
M = mean(A.T, axis=1)
C = A - M
V = cov(C.T)
values, vectors = eig(V)
P = vectors[0:3].dot(C.T)
P.T.shape

In [ ]:
train_data = train_data.drop('Sex',axis = 1)

In [ ]:
Ytrain_data = train_data['Age']

In [ ]:
train_data = train_data.drop('Rings',axis = 1)
train_data = train_data.drop('Age',axis = 1)

# Decision tree

In [ ]:
def gini_index(groups, classes):
    all_samples = float(sum(len(group) for group in groups))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        probability = 0.0
        if size == 0:
            continue
        for class_index in classes:
            p = [row[-1] for row in group].count(class_index) / size          
            probability += p * p
        gini += (1 - probability)* size/all_samples
    return gini

In [ ]:
def entropy_index(groups, classes):
    all_samples = float(sum(len(group) for group in groups))
    entropy = 0.0
    for group in groups:
        size = float(len(group))
        probability = 0.0
        if size == 0:
            continue
        for class_index in classes:
            p = [row[-1] for row in group].count(class_index) / size   
            if p != 0:
                probability += p * np.log(p)
        entropy -=  probability * size/all_samples
    return entropy

In [ ]:
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right            

In [ ]:
def get_split(dataset, measurement):
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    class_values = list(set(row[-1] for row in dataset))
    features = list()
    while len(features)<3:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            if measurement == 1:
                score = gini_index(groups, class_values)
                #print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], score))
            else:
                score = entropy_index(groups, class_values)
                #print('X%d < %.3f Entropy=%.3f' % ((index+1), row[index], score))
            if score < b_score:
                     b_score, b_index, b_value, b_groups = score, index, row[index], groups
    return {'gini':b_score, 'index':b_index, 'value':b_value, 'groups':b_groups}

In [ ]:
def terminal_outcome(node):
    outcomes = [row[-1] for row in node]
    return max(set(outcomes), key = outcomes.count)

In [ ]:
def split(max_depth, min_size, node, depth, measurement):
    left, right = node['groups']
    del node['groups']
    if not left or not right:
        node['left'] = node['right'] =  terminal_outcome(right + left)
        return
    if depth >= max_depth:
        node['left'], node['right'] = terminal_outcome(left), terminal_outcome(right)
        return
    if len(left) <= min_size:
        node['left'] = terminal_outcome(left)
    else:
        node['left'] = get_split(left, measurement)
        split(max_depth, min_size, node['left'], depth+1, measurement)
    if len(right) <= min_size:
        node['right'] = terminal_outcome(right)
    else:
        node['right'] = get_split(right, measurement)
        split(max_depth, min_size, node['right'], depth+1, measurement)    

In [ ]:
def build_tree(max_depth, min_size, measurement, dataset):
    root = get_split(dataset,measurement)
    split(max_depth, min_size, root, 1, measurement)
    return root

In [ ]:
def print_tree(node, depth = 0):
    if isinstance(node, dict):
        print('%sX%d < %.3f' % (depth*' ', node['index']+1, node['value']))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s%d' % (depth*' ', node))

In [ ]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [ ]:
def prediction(tree,test_data):
    predictions = list()
    for row in test_data:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return predictions

In [ ]:
def accuracy(actual, predict):
    correct = 0.0
    for row in range(len(predict)):
        if actual[row] == predict[row]:
            correct += 1
    return float(correct)/len(predict) * 100

In [ ]:
def decision_tree(train_data,test_data, max_depth, min_size, measurement):
    actual_train = list()
    actual_test = list()
    tree = build_tree(max_depth, min_size, measurement,train_data)
    print_tree(tree)
    predictions_train = prediction(tree,train_data)
    predictions_test = prediction(tree,test_data)
    for row in train_data:
        actual_train.append(row[-1])
    for row in test_data:
        actual_test.append(row[-1])
    accuracy_train = accuracy(actual_train, predictions_train)
    accuracy_test = accuracy(actual_test, predictions_test)
    return accuracy_train, accuracy_test, tree

In [ ]:
Xtrain = P.T
C = train_data[3133:4177] - M
P = vectors[0:3].dot(C.T)
Xtest = P.T
Ytrain = Ytrain_data[0:3133]
Ytest = Ytrain_data[3133:4177]

In [ ]:
len(Xtrain)

In [ ]:
Ytrain = Ytrain.values
Xtrain_my = np.column_stack((Xtrain, Ytrain))
Ytest = Ytest.values
Xtest_my = np.column_stack((Xtest, Ytest))

In [ ]:
accuracy_train, accuracy_test, tree = decision_tree(Xtrain_my, Xtest_my, 4, 2, 1)

In [ ]:
p = np.arange(0., 1., 0.01)
y = 2*p*(1-p)
yy = (-p*np.log(p)-(1-p)*np.log(1-p))*0.7246
pl.plot(p,y,p,yy)
pl.text(0.8, 0.2, 'Gini')
pl.text(0.8, 0.4, 'Entropy')
pl.title('Node impurity measurements for two-class classification')
pl.show()

In [ ]:
def subsample(dataset, ratio):
    sample = list()
    total = round(ratio * len(dataset))
    while len(sample) < total:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [ ]:
def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key = predictions.count)

In [ ]:
def bagged_tree(train, test, ntrees, max_depth, min_size, measurement):
    trees = list()
    for n in range(ntrees):
        sample = subsample(train, 1)
        tree = build_tree(max_depth, min_size, measurement, sample)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return predictions

In [ ]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    size = int(len(dataset)/n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) <  size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [ ]:
def evaluate_cv(dataset, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set,[])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            #row_copy[-1] = None
            test_set.append(row_copy)
        actual = [row[-1] for row in fold]
        predictions = bagged_tree(train_set, test_set, *args)
        accuracy_test = accuracy(actual, predictions)
        scores.append(accuracy_test)
    return scores

In [ ]:
def evaluate(trainset, testset, *args):
    train_set = list()
    test_set = list()
    scores = list()
    for row in trainset:
        row_copy = list(row)
        #row_copy[-1] = None
        train_set.append(row_copy)
    for row in testset:
        row_copy = list(row)
        #row_copy[-1] = None
        test_set.append(row_copy)
    actual = [row[-1] for row in testset]
    predictions = bagged_tree(train_set, test_set, *args)
    accuracy_test = accuracy(actual, predictions)
        #scores.append(accuracy_test)
    return accuracy_test

In [ ]:
train_data_my = np.column_stack((train_data.values, Ytrain_data.values))

In [ ]:
for n in [1,5,10]:
scores = evaluate(Xtrain_my.tolist(), Xtest_my.tolist(), n, 4,2,1)
print('Accuracy: %f' % scores)
    #print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

# Regression

In [ ]:
train_data = train_data.drop('Sex', axis = 1)
Ytrain_data = train_data['Age'][0:3133]
Ytest_data = train_data['Age'][3133:4177]
train_data = train_data.drop('Age', axis = 1)
train_data = train_data.drop('Rings', axis = 1)
Xtrain_data = train_data[0:3133]
Xtest_data = train_data[3133:4177]

In [ ]:
len(Xtest)

In [ ]:
regr = DecisionTreeRegressor(max_depth=4)
regr.fit(Xtrain, Ytrain)
predict = regr.predict(Xtest)

In [ ]:
absmin = min([Ytest.min(),predict.min()])
absmax = max([Ytest.max(),predict.max()])
ax = pl.axes()
ax.scatter(Ytest,predict)
ax.set_title('Y vs. YHat Regression tree')
ax.axis([absmin, absmax, absmin, absmax])
ax.plot([absmin, absmax], [absmin, absmax],c="k")
mae = mean_absolute_error(Ytest_data, predict)
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
textStr = 'MAE=%.3f' % mae
ax.text(0.05, 0.95, textStr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)


In [ ]:
lasso_model = linear_model.LassoCV(cv=5).fit(Xtrain, Ytrain)
ypred2 = lasso_model.predict(Xtest)

In [ ]:
absmin = min([Ytest.min(),ypred2.min()])
absmax = max([Ytest.max(),ypred2.max()])
ax = pl.axes()
ax.scatter(Ytest_data,ypred2)
ax.set_title('Y vs. YHat Lasso(CV=5)')
ax.axis([absmin, absmax, absmin, absmax])
ax.plot([absmin, absmax], [absmin, absmax],c="k")
mae = mean_absolute_error(Ytest_data, ypred2)
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
textStr = 'MAE=%.3f' % mae
ax.text(0.05, 0.95, textStr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)

